# Redes recurrentes

Proyecto PAPIME PE102723

Esta red aprenderá un patrón subyacente a los nombres que se les dan a los dinosaurios, utilizaremos esto para pedirle que genere nombre nuevos.

## Datos
Usaremos los nombres de dinosarios descargados de [este repo](https://github.com/codificandobits/Generacion_de_nombres_con_Redes_Recurrentes/blob/master/nombres_dinosaurios.txt), incluímos aquí ya la copia por practicidad.  Este ejercicio se basa en [Tutorial: generación de texto con Redes Recurrentes en Python](https://www.codificandobits.com/blog/tutorial-generacion-de-texto-con-redes-recurrentes-python/).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from ipywidgets import interact, interact_manual
import ipywidgets as widgets

In [2]:
import torch
from torch import nn

In [3]:
# Path for saving the network
PATH = 'data/dinos.pth'

In [4]:
# Se carga todo el archivo como una sola cadena muy larga
nombres = open('data/nombres_dinosaurios.txt','r').read()
# nombres

In [5]:
# Entradas: caracteres

alfabeto = ['.'] + sorted(list(set(nombres)))
tam_datos, tam_alfabeto = len(nombres), len(alfabeto)

print("Número de entradas posibles:", tam_alfabeto)
print("Número de ejemplares:", tam_datos)
print("Entradas:", alfabeto)

Número de entradas posibles: 54
Número de ejemplares: 19909
Entradas: ['.', '\n', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [6]:
# Mapeo de caracter a posición
car_a_ind = { car:ind for ind,car in enumerate(alfabeto)}

In [7]:
def one_hot(car):
    """
    Devuelve la codificación one hot para el caracter según el
    alfabeto de este problema
    """
    ceros = np.zeros(tam_alfabeto)
    ceros[car_a_ind[car]] = 1
    return ceros

In [8]:
def a_car(one_hot):
    """
    Devuelve el caracter representado por el vector one hot.
    """
    índice = np.argmax(one_hot)
    return alfabeto[índice]

In [9]:
@interact(index = (0, tam_alfabeto - 1) )
def test_one_hot(index):
    c = alfabeto[index]
    print("Caracter =", c)
    return one_hot(c)

interactive(children=(IntSlider(value=26, description='index', max=53), Output()), _dom_classes=('widget-inter…

In [10]:
# Separación de palabras

nombres_separados = nombres.split('\n')
nombres_separados = ['.' + nom for nom in nombres_separados]  # . indica el inicio
print('Ejemplo del formato de entrada:')
nombres_separados[0]

Ejemplo del formato de entrada:


'.Aachenosaurus'

In [11]:
# Entradas

X_list = []
for nombre in nombres_separados:
    lm = []
    for c in nombre:
        lm.append(one_hot(c))
    m = np.vstack(lm)
    X_list.append(m)
# X_list[0]    # (L,H_{in})

In [12]:
# Salidas, los caracteres se recorren uno a la izquierda

Y_list = [np.vstack((m_nombre[1:], one_hot('\n'))) for m_nombre in X_list]
#Y_list[0]

In [13]:
@interact(index = (0, len(X_list) - 1) )
def test_XY(index):
    print([a_car(oh) for oh in X_list[index]])
    print([a_car(oh) for oh in Y_list[index]])

interactive(children=(IntSlider(value=767, description='index', max=1535), Output()), _dom_classes=('widget-in…

In [14]:
# Cada palabra tiene una longitud distinta
X = np.array(X_list, dtype=object)  # Arreglo de objetos
Y = np.array(Y_list, dtype=object)

t_indices = np.arange(len(X_list))
np.random.shuffle(t_indices)
t_i1 = int(len(X) * 0.7)
i_train = t_indices[:t_i1]
i_test = t_indices[t_i1:]

# Entrenamiento
X_train = X[i_train]
Y_train = Y[i_train]

# Prueba
X_test = X[i_test]
Y_test = Y[i_test]

#X_train

# Red recurrente

Para cada elemento en la secuencia, cada capa calcula:
\begin{align}
  h_t = \tanh(x_t W_{ih}^T + b_{ih} + h_{t-1} W_{hh}^T + b_{hh})
\end{align}
con $x_t$ la entrada y $h_t$ el estado oculto al tiempo $t$, $h_(t-1)$ es el estado oculto de la capa anterior al tiempo $t-1$ o el estado oculto inicial.

In [15]:
NUM_OCULTAS = 25
NUM_CAPAS_RNN = 1
BIDIRECCIONAL = False
D = 2 if BIDIRECCIONAL else 1
class Dinosauriólogo(nn.Module):
    """
    Red recurrente para generar nombres de dinosaurios.
    """
    def __init__(self, n_entrada=tam_alfabeto, n_oculta=NUM_OCULTAS):
        super(Dinosauriólogo, self).__init__()
        self.rnn = nn.RNN(input_size=n_entrada,
                          hidden_size=n_oculta,
                          num_layers=NUM_CAPAS_RNN,
                          nonlinearity='tanh',
                          bias=True,
                          bidirectional=BIDIRECCIONAL)
        self.ffn = nn.Linear(n_oculta, tam_alfabeto, bias=True)
        self.smax = nn.Softmax(dim=1)
        
    def forward(self, x):
        salida, oculta = self.rnn(x)  #h_0 son ceros por defecto
        salida = self.smax(self.ffn(salida))
        return salida

In [16]:
RR_0 = Dinosauriólogo()

## Uso

In [17]:
def nombre_a_cadena(narr):
    """
    Obtiene el nombre codificado en la matriz de vectores one hot.
    """
    return ''.join([a_car(oh) for oh in narr])

In [18]:
# https://pytorch.org/docs/stable/generated/torch.nn.RNN.html#torch.nn.RNN
# Estado oculto inicial (1, H_{out})
@interact(index = (0, len(i_train) - 1) )
def predicción(index):
    # Ejemplar (L,H_{in})
    x = torch.tensor(X_train[index], dtype=torch.float32)

    # Evaluación de la red
    salida = RR_0(x)
    
    print(nombre_a_cadena(X_train[index]),
          '->',
          nombre_a_cadena(salida.detach().numpy()))

interactive(children=(IntSlider(value=537, description='index', max=1074), Output()), _dom_classes=('widget-in…

# Entrenamiento

In [19]:
def train(model, X, Y, X_test, Y_test, learning_rate=0.01, momentum=0.5, weight_decay=0, num_steps=500, alpha=0.75):
    """
    Recibe el modelo de red neuronal a entrenar,
    los datos de entrada X y los valores de salida deseados Y
    en tensores de PyTorch
    """
    errores = np.zeros(num_steps)
    errores_test = np.zeros(num_steps)
    
    #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, dampening=0, weight_decay=weight_decay, nesterov=True)
    #optimizer = torch.optim.ASGD(model.parameters(), lr=learning_rate, lambd=0.0001, alpha=alpha, weight_decay=weight_decay) # Averaged Stochastic Gradient Descent
    #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, alpha=0.99, weight_decay=weight_decay, momentum=0.5) # Adaptive lr
    #optimizer = torch.optim.Adadelta(model.parameters(), lr=1.0, weight_decay=weight_decay) # Adaptive lr
    #optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01,lr_decay=0,weight_decay=weight_decay) # Adaptive subgradient
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    #criterion = torch.nn.MSELoss()       # Mean Squared Error
    #criterion = nn.CrossEntropyLoss()
    criterion = nn.BCELoss()
    
    for t in range(num_steps):
        
        # Entrena
        for i in range(len(X)):
            # Forward pass: Compute predicted y by passing x to the model
            y_pred = model(X[i])

            # Compute and print loss
            loss = criterion(y_pred, Y[i])
            errores[t] += loss.item()
            #print(loss, ":", y_pred, Y[i])

            # Zero gradients, perform a backward pass, and update the weights.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # Prueba
        for i in range(len(X_test)):
            # Forward pass: Compute predicted y by passing x to the model
            y_pred = model(X_test[i])

            # Compute and print loss
            test_loss = criterion(y_pred, Y_test[i])
            errores_test[t] += test_loss.item()
    if num_steps > 1:
        # Grafica error
        steps = np.arange(num_steps)
        plt.plot(steps, errores, label='Entrenamiento')
        plt.plot(steps, errores_test, label='Prueba') # A veces estorba
        plt.legend()
        plt.title("Error")
        plt.ylabel("Error" + str(optimizer))
        plt.xlabel("Iteración")
    print("Final error = ", errores[-1])

In [20]:
num_clicks = 0
@interact_manual()
def mini_entrena():
    torch_X_train = [torch.tensor(x, dtype=torch.float32) for x in X_train]
    torch_Y_train = [torch.tensor(y, dtype=torch.float32) for y in Y_train]
    torch_X_test = [torch.tensor(x, dtype=torch.float32) for x in X_test]
    torch_Y_test = [torch.tensor(y, dtype=torch.float32) for y in Y_test]
    train(RR_0, torch_X_train, torch_Y_train, torch_X_test, torch_Y_test,
          learning_rate=0.01, num_steps=50)
    global num_clicks
    num_clicks += 1
    print("Run:", num_clicks)

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

In [21]:
@interact_manual()
def save_network():
    torch.save(RR_0.state_dict(), PATH)

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

In [22]:
@interact_manual()
def load_network():
    global RR_0
    RR_0 = Dinosauriólogo()
    state_dict = torch.load(PATH)
    RR_0.load_state_dict(state_dict)
    print("Se cargó el archivo con los pesos de la red")

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

# Producción

Se utiliza la red para producir el caracter por caracter.  Se alimenta inicialmente con '.' y luego se utiliza su salida como entrada siguiente hasta que produzca el caracter '\n' o haya alcanzado el número límite de caracteres.

In [23]:
@interact
def produce():
    # Ejemplar (L,H_{in})
    x = torch.tensor(one_hot('.').reshape(1,tam_alfabeto), dtype=torch.float32)
    num = 0
    dino = ''
    c = ''
    
    while c != '\n' and num < 20:
        # Evaluación de la red
        salida = RR_0(x)
        c = nombre_a_cadena(salida.detach().numpy()) 
        dino += c
        
        print(nombre_a_cadena(x), '->', c)
        x = torch.tensor(one_hot(c).reshape(1,tam_alfabeto), dtype=torch.float32)
        num += 1
    print("Dino se llama ", dino)

interactive(children=(Output(),), _dom_classes=('widget-interact',))